# IaaS and IaaC

Infrastructure   
1. Compute consists of VMS such as EC2, Containers such as Docker
2. Storage consists of Object Storage (S3), Non-relational Stores (DynamoDB), and Relationsl Stores (Amazon RDS, Google Cloud SQL)
3. Networking consists of Virtual Private Cloud (VPC), Load Balancers (AWS ELB)  


As a service model consists of 
1. A web service accessible over HTTP
2. Typically exposing APIs with client side tools including CLIs, SDKs, UI
3. Multi-tenant- same service is able to support multiple users
4. Pay as you use charging model

IaaS- web service that supports resource management   
Provisioning and Management includes Creating, updating, and managing resources. Infrastructure provisioning in the real world is done through tools that support a declarative model for provisioning and management

### AWS CloudFormation

Create a template describing the infrastructure resources you want in a YAML/JSON format (this is a declarative definition)  
Give the template to the CloudFormation web service, which takes care of provisioning all infrastructure resources defined in the template.   
Technically Infrastructure as Code, which is a web service that works with infrastructure defined in this declarative format.  
You express what is required, not how to do it.

### Advantages of "as-Code"

1. Easier to understand what is being provisioned. With a block of code, it's hard to easily see what is being provisioned.
2. Robust provisioning of Infrastructure, if a single resouce fails, it will restart
3. Infrastructure definition templates can be shared with others.
4. Easy to recreate defined infrastructure
5. Easy to track infrastructure resource templates

High level abstractions with Infrastructure resource (Look at first paragraph)  
Target user is person managing architecture, skills needed is understanding of resources and scripting.

### AWS CloudFormation (again)

Abstraction is resources, templates is the formatted file that defined the resources needed, stack is the set of resources provisioned.   

### What is Declared?

Resources with different properties, input/output parameters. IAM permissions are **not declared**

In order to make a template reusable, CloudFormation uses input parameters and mappings. IAM has Policy Variables.  
In order to define ordering between different resources, it is either explicitly specified or implicitly defined (through an attribute).  
All actions are supported (CRUD)  
Operations can be All or nothing (either creates or doesn't, and if some action fails, then you delete all previously created stuff) and partially created

### Resources

Top level object in CloudFormation template:  
Resources: 
HelloBucket (Logical Name)  
Type: AWS::S3::Bucket   
Logical name is the name used in the template, physical name is generated by AWS and is a conbination of Logical name, stack name, and unique ID.  
Resource properties are specified with the resource like such:   
HelloBucket (Logical Name)  
Type: AWS::S3::Bucket  
Properties:   
AccessControl: PublicRead  
In addition, you can have multiple properties, and each property can have multiple values

### Resource Attributes

CreationPolicy: Attribute when resource prevents status from reaching create complete until CloudFormation receives number of success signals or until timeout is exceeded.   
DeletionPolicy: Attribute that allows you to preserve the state of a resource before it is deleted, done through retain policy *DeletionPolicy: Retain*   
DependsOn - attribute that allows you to specify whether you want the creation of an attribute to follow another i.e. *DependsOn: myDB*.  
Metadata- attribute that associated structured data with resource.  
Intrinsic functions provided by AWS, use these in your templates to assign values to properties not available until runtime. Can be used in resource policies, outputs, metadata attributes, update policy attributes   
Ref- in built function that takes in the logical name, outputs the physical name   
GetAtt- gets the attributes of resource, takes in logical name of resource and name of attribute, outputs attribute value.  

Parameter definition consists of type of parameter, contraints on parameter value, default value

# Platform and PaaS

Platform is a set of one or more infrastructure resources configured to run specific apps (i.e. Python web platform, EC2 instance configured with Python2.7 and Nginx)

### PaaS

PaaS is a web service that applies transformations on infrastructure resources to generate the required plaform, also deploy apps on the generated platform,

### IaaS vs PaaS

IaaS mostly just creates the infrastructure, user then has to deal with the application deployment, but PaaS does deployment on top of infrastructure

You can do some of the PaaS stuff with IaaC. IaC could be good for deploying pre-built static apps, but other than that it's not good for deployment. Normally, it's used by a devops person, not for app devs.

IaaC is great for pre-built apps that never change, but if it is under constant change, IaC does not have the ability to build application from source code.

Using PaaS, app developers can do production applicatino deployment. This has led to DevOps engineers

### DevOps

Interpretation 1: It is a functional role in which they occasionally perform Operations duties.  
Interpretation 2: It is a set of steps/processes, and these engineers use development practices when setting up infrastructure.

App developers need to be aware of infrastructure, and should not treat it as a black box. Actual infrastructure work is still done by Operations Engineers.   
Operations Engineers need to practice good development practices and closely work with app devs to determine production architecture and required infrastructure

### PaaS features

Allows you to build/deploy applications, provides a load balancer, DNS name connected to load balancer, ability to configure SSL Certs, scales application (manually or triggered by policies), canary deployment, and deployment without any downtime, rollbacks, and log collection

# Web apps

Web pages - static. 
Web applications - dynamic   
Web sites - web pages + web applications.  
Primary characteristics are dynamic content, persistent storage, API access. Uses HTTP, frameworks to make it, and is logically divided into three layers (HTML UI, Web App request handling, server logic, data access layer to DB)

# EBS

Amazon's PaaS, deploys web apps through source code to EC2, and beanstalk handles EC2 provisioning.   
Application can use other AWS resources, but Beanstalk provisions a LoadBalancer for the deployed application and the app can provision a routable DNS record

1. Application: OS folder that contains the app source code.  
2. Application Version: Specific version of the application code, stored on S3
3. Environment: Application version deployed on a set of AWS resources, each environment runs a single application version
4. Environment Tier: Tier determines whether the resources provisioned by Beanstalk would be used for deploying a backend or a web app that handles HTTP traffic
5. Environment Configuration: determines prameters and settings of environment's resources

### Host Manager

Beanstalk software that runs on each EC2 instance in an environment, has to deploy app, aggregate events, generate events, monitor logs, monitor app server, patch instance components, publish logs to S3

Environment types:
1. Load balancing/autoscaling
2. Single Instance

### IAM Permissions

User policies: permissions that your IAM user needs to interact with EBS.   
Service Role: Needed by EBS to interact with other AWS services on your behalf.  
Instance Profile: Role and permissions granted to EC2 instance in your environment

### User Policy

EBS will use the permissions given to the IAM user when creating other AWS resources, user needs FullAccess managed Policy, EBS uses user permissions to launch all resources in environment

### Service Role

Contains all nevessary permissions that allows EBS to call other AWS services without user intervention, permissions that allow EBS to monitor instance and environment health and update nevironments to perform updates. A service role is created by default when creating an environment

### Instance Profile

AWSElasticBeanstalkWebTier - Policy Statement that allows EC2 instantces in your environment to upload logs to S3, default instance profile is aws-elasticbeanstalk-ec2-role

### Platform/Container type

Software stack to run on EC2 instances provisioned as part of an environment is determined by the platform/container type of environment. This is different from Docker. We're talking about Apache Tomcat here.

### PaaS criticisms

1. Not able to change software stack used by application. This is changing with Docker
2. Less control, not possible to control configuration and setting of IaaS resources. 
3. Once you use a PaaS on one vendor, it's hard to migrate to other PaaS
4. It's costly and difficult to use when apps are being developed

All AWS resources are provisioned as a part of an environment that can be customized using a config file in the app folder. Things that can be customized  include the software stack settings on the EC2 instances, type and flavor of provisioned resources.   
.ebextensions includes files with config files. 
Use services to define the services/processes you want started on your EC2 instances. Container commands are run after the app/web server have been set up and the application version has been extracted.   
You can use CloudFormation snippets to customize AWS resources. This is unique to EBS. EBS can also create an RDS instance, and EBS will provide the data in environment variables that needs to be read in.

# Docker

Docker solves the problem that is it should be possible to repeatedly deploy an application whilst always getting the same result.   
This is important, because any app that is developed and tested locally should work exactly like that in production. Applications should behave the same across different environments

### Repeatability Issues

1. It's hard to ensure that there's similarity between environment components across environments. This is usually solved by PaaS or IaC for deployment
2. Consistency of packages and libraries across different environments is hard. This has been solved by packages, but OS-level packages is harder, so the way you do it is by packaging OS along with the app

Challenges with packaging OS is it's hard to define what OS we want, what the end result of the packaging would look like, where the App + OS would run, how to bind it to databases, and how to access the app

### Utilization Issues

Modern Hosts have large capacity, so containers allow us to maximize the utilization of a host

### Resource Isolation

We need to isolate the Dockerized OS with the host OS. Requirements are:   
1. Root file systems of Containers should remain separate
2. Processes that are running inside Containers should remain separate
3. Network traffic destined for different Containers should remain separate
4. A rouge container should not be able to affect other containers running on the host

### Docker

Docker is a technology for packaging OS + Application.   
It started as a technologu differentiator within PaaS called DotCloud.   
Core Docker technology open sourced in 2013, renamed to Docker Inc.

### Dockerfile

Declarative definition format that defined Base OS, any packages, where to inject app, what environment variable to set, which port the app will listen to

### How does Docker work?

Docker daemon pulls down base OS image specified in the Dockerfile, which is located in the Dockerfile in the root directory.  
Docker daemon executes each line defined in the Dockerfile on this base image. This daemon builds the final image in the form of layers, and each line in the Dockerfile causes a new diff layer to be generated over the previous layer

### Basic Docker Commands

Build a docker container is `docker build -t <tag-name>`. Dockerfile needs to be present in the directory from which this command is executed   
Execute a docker container is `docker run -d -p <host-port>:<container-port><tag-name>` -d runs Docker in detached mode, -p maps container port to host port, tag-name is the name used in the `docker build` command.   
To see logs, you run `docker logs <container-id>`.   
To see all running Docker containers `docker ps`.  
To stop a docker container `docker stop <container-id>`   
To remove a stopped Docker container `docker rm <container-id>`.   
To remove a docker container image run `docker rmi <tag-name>`

# Container Orchestration

There are a lot of container orchestration systems, which allow you to run containers across a number of hosts.

### Omega

Global cluster state whith multiple schedulers that maints a local copy of the global state. Each scheduler can make independent scheduling decisions. If some resource is being used by some other task, they are preempted and given to the requesting scheduler depending on job priority.

Lessons learned is that you should allocate separate IP addresses for each container (don't separate container URLs based on port numbers allocated by the orchestration engine), Use a higher level grouping mechanism for grouping containers. When containers corresponding to one app are running together, we need a way to address them together. Kube uses Labels and label selectors and containers can be assigned labels that can be the target of operations

# Containers and Managed Services

The problem is how to bind app containers with managed services. The application can start interacting with the managed service after binding is successful.

### Issue with Binding

How do you specify Managed Service's Connection details to the applicatino container? This is done through environment variables

### Issues in Binding: Authorization

Container Authorization: Granting permissions to your application containers to perform the required action on the managed instance   
Required permissions will depend on the type of the data store and the specific actions that your application containers need to take

### Issues in Binding: Traffic Restriction

Traffic Restriction- restricting network that can reach managed data store instance   
You should only allow traffic that originates from the cluster where your application containers are running to reach the managed instance. Traffic from any other sources should be disallowed.

Approach #1 is to manually give permissions to App container, and modify Firewall to receive traffic from container cluster

Approach #2 is to automate the manual approach and modify firewall/security group of Managed Service to receive traffic only from container cluster

Approach #3 is sidecar proxy container, so run a container that acts as a proxy to Managed Service, give permissions to proxy container, app communicates with proxy, and Modify firewall/security group of Managed Service instance to receive traffic only from Container cluster

# GCP

IAM enables granting access to GCP (determines who has what role for each resource).  
Who refers to Google account, service account   
Resources refers to project, compute engine instance, cloudSQL instances.  
Project is the top level resource from which other resources are created

### Permissions

Determines what operations are allowed on a resource, represented as $<service>, <resource>, <verb>$, and corresponds with REST actions on a resource. Caller of REST action needs those same permissions

### Roles

Role is a collection of permissions and cannot be assigned to users directly. Users gain permission through roles. Primitive roles are owner, editor, viewer. There are also predefined roles that give fine grained permissions that primitive roles cannot satisfy. Custom roles are roles that do not fit predefined and primitive roles, that you need to satisfy your needs.

### GCP Policy Hierarchy

Policy can be defined at any level in Resource hierarchy. 
Resources inherit the policies of parent resource.   
If a policy is set at project level, it is inherited by all its child resources.   
Child policies cannot restrict access granted at a higher level.   
Effective policy at a resource = union of all policies defined at the resource and inherited policies

### Configuring User Access

Manual Setup is gcloud auth login, which will open a browser window to grant permission to gcloud cli to connect to your account, token only valid for an hour.   
Can also be configured through Service Account, in which you grant roles to it. This gives you a service account file, to which you configure gcloud CLI to use that file.   
You can also use automated Renewal like CaaStle

# Microservices

Traditional application is monolithic, all functionality in same code. This makes it super easy to develop and deploy, but the cons is that it has to be devloped by the same team in the same language, and it's hard to evolve one functionality.   
Microservice Architecture is where you break up application functinoality into separate services accessible over HTTP.   
Only real con is overhead when beginning development.

### Why Microservices?

Containerization makes it easy to package different application components, and Kubernetes makes it possible to run and scale containers.

### Challenges with Microservices

Serivces discover and bind to one another through a service target port pair defined (this is in Kube).   
If a service is failing in the architecture, you need to have robust programming of each service to assume failures could occur.   

### Meta points

No way to define microservice boundaries, really depends on the app.   
You usually start with a monolithic app, and you may transition this over to a microservice architecture

# PaaS vs CaaS

### Similarities

1. Run/Deploy application, with the ability to rollback
2. Bind applications to managed services like DBs
3. Scale application instances
4. Routing of outside requests to appropriate application instances
5. Load balancing of incoming traffic

### Differences

1. PaaSes deploy application from application source code, whereas CaaSes deploy application containers
2. PaaSes typically provision managed service instances and then bind the application to it, whereas CaaSes mostly focus on application container deployment, but provide a mechanism for binding
3. PaaSes lack good local development support   
4. PaaS is more suited to monolithic stuff, whereas CaaS is suited to both microservice and monolithic
5. PaaSes provide end to end functionality, but CaaSes require you to assemble the pieces

# Kubernetes

Container Orchestration System developed by Google written in Golang

### Kubernetes Control Loop

Asks what the desired/current state is, checks if these two is the same. If not, then make changes to make them the same (known as reconciliation action).   
All Kubernetes Controllers execute this control loop.  

Basic Objects/Abstractions: Pod, Service, Volume, Namespace

### Pod

Pod represents a unit of deployment, which contains an application container, storage resources required by the container, unique network IP address, options that govern how the container should run

### Pod Creation

`kubectl create -f greetings.yaml` This command is executed on the user machine, adn the request is received by the apiserver, which stores the YAML in etcd, which is a key value store for Google.    
The Apiserver hands over request ot the controller manager, which runs the control loop to create the pod (basically desired state is a new pod needs to be created, it doesn't exist, so create the pod)

### Service

Service defines a set of pods and a policy to access them. The set of pods target is determined by a LabelSelector. Although each pod has a unique IP address, they are not exposed unless you have a service to receieve traffic.

Service can be exposed in different ways by specifying a type in the ServiceSpec:   
ClusterIP: Exposes a service on an internal IP in the cluster. This service is only reachable from within the cluster.  
NodePort: Exposes the service on the same port of each selected node in the cluster. From the outside, the service is accessbiel using a NodeIP:NodePort pair.   
LoadBalancer: Creates an external load balancer in the underlying cloud and assigns a fixed, external IP of the Service.   
ExternalName: Exposes the service using an arbitrary name

### Deploying a container?

Option 1: Create a Pod and expose it using a service. Need a separate file for service and pod. Problem with this is that you need to manage two files, and keep track of label and selector with two different files. It's hard to create more than one pod, deploy new versions, and revert versions.   
That's where the deployment controller comes in.

### Deployment Controller

A deployment controller creates a declarative model for creating/updating pods. It's a way to expose Pods to the outside world.   
You would use the deployment controller to declare/create new pods, scale up deployment to handle load, rollback to an earlier deployment, pause the deployment  

Option 2 for deployment: Use a deployment object. The deployment controller will create a service object for exposing the Pods defined in the deployment YAML

### CICD using Kube

Problem is that deployments change the IP address of the app. Keeping IP address is crucial for CICD to do stuff. The solution is to use a kubernetes ingress object, which creates a static IP address for our application

### Namespaces

It's a mechanism that allows us to use the same Kube cluster for running Pods, Services, Deployments for different applications. Users interacting with one namespace do not see the content in another namespace

### Labels

Labels are a mechanism for selecting and accessing Kubernetes objects, and is ued by controllers for performing the reconciliation actions as a part of the Control loop. 

### Volumes

It's a mechanism that allows pods to store data in a filesystem. A volume is a directory accessible to containers in a pod. A volume is terminated/deleted when the corresponding pod is terminated

# Cloud Revisited

Enterprises are adopting CloudComputing, because there's no investment in hardware resources, more secure, and easier/quicker to adopt agile and new tech. It's also available across the globe.

Enterprises are going for SaaS, which is a model that means you have software on the internet that tpeople can access

### Concerns

1. Vendor lock-in (once a specific cloud is chosen, it's hard to change)
2. Migration (how to move exiting apps to the cloud)
3. Cloud Expertise (how to best utitlize the cloud)

### Migration

Vendor lock-in - use multiple clouds and OpenStack, which is open source cloud software.   
Migration - invest in tools for cloud migration, don't migrate everything

Orgs build their own data centers for control, cheaper cloud costs, unique requirements

People are moving to multi-cloud, where you have public and private clouds. You can use tools that support Multi-Cloud
1. FirstMile - Webapp deployment to ebs and Google App Engine
2. Caastle - Containerized deployment on GKE and ECS
3. CloudFoundry- PaaS that supports multiple clouds
4. Terraform- IaS for different cloud providers
5. Gravitant/IBM - Cloud Broker software

More people using public clouds, less people using hybrid clouds, Docker used broadly, Kube use is going down, private cloud is going up

Private cloud is AWS/GCP setup in your own data center. Many tools such as vSphere, Microsoft System Center, OpenStack

### OpenStack

Software for creating private and public clouds, code written in python and available on GitHub

# CICD